In [1]:
import tornado.ioloop
import tornado.web
import tornado.concurrent

from time import time

from sockjs.tornado import SockJSConnection, SockJSRouter, proto

from google.transit import gtfs_realtime_pb2
import urllib



class IndexHandler(tornado.web.RequestHandler):
    """Regular HTTP handler to serve the ping page"""
    def get(self):
        self.render('index.html')
        



# # Out broadcast connection
# class BroadcastConnection(SockJSConnection):
#     clients = set()

#     def on_open(self, info):
#         self.clients.add(self)

#     def on_message(self, msg):
#         self.broadcast(self.clients, msg)

#     def on_close(self):
#         self.clients.remove(self)

# BroadcastRouter = SockJSRouter(BroadcastConnection, '/broadcast')


# Stats class
class StatsConnection(SockJSConnection):
    
    clients = set()
    
    last_update = time()
    
    gisdata = {}
    
    buses = 0
    
    def on_open(self, info):
        self.clients.add(self)
        self.loop = tornado.ioloop.PeriodicCallback(self._send_stats, 3000)
        self.loop.start()

    def on_message(self):
        pass

    def on_close(self):
        self.clients.remove(self)
        self.loop.stop()

    def refresh(self):
        print "refreshing"
        feed = gtfs_realtime_pb2.FeedMessage()
        response = urllib.urlopen('https://data.texas.gov/api/file_data/cxEUVbzwHvy6_WDMKITWyEe9fT8QQxXMKu2YwJBTKTs?filename=VehLoc.pb')
        feed.ParseFromString(response.read())
        counter= 0
        geobase = { "type": "FeatureCollection",
                        "features": []
                         }
        for entity in feed.entity:
          if entity.HasField('vehicle'):
            featurebase = { 
                "type": "Feature",
                "geometry": {"type": "Point", "coordinates": [float(entity.vehicle.position.longitude), 
                                                              float(entity.vehicle.position.latitude)]},
                "properties": {"id": entity.vehicle.vehicle.id}
            }
            geobase['features'].append(featurebase)
            counter +=1
        self.buses = counter
        self.gisdata = geobase
        

    def _send_stats(self):
        #update every 5 seconds
        #print "here is the last update", self.last_update, "current", time(), "and diff", time()-self.last_update
        if time()-self.last_update > 3:
            self.refresh()
        data = proto.json_encode({"cusnum":len(self.clients),
                                  "busnum":self.buses,
                                 "geodata":self.gisdata})
        self.send(data)

StatsRouter = SockJSRouter(StatsConnection, '/statsconn')

if __name__ == "__main__":
    import logging
    logging.getLogger().setLevel(logging.DEBUG)

    # Create application
    app = tornado.web.Application(
            [(r"/", IndexHandler)] +
            StatsRouter.urls
    )
    app.listen(9999)

    print 'Listening on 0.0.0.0:9999'

    tornado.ioloop.IOLoop.instance().start()

Listening on 0.0.0.0:9999


RuntimeError: IOLoop is already running

INFO:tornado.access:200 GET /statsconn/info (::1) 1.00ms


refreshing
refreshing
refreshing
refreshing
refreshing
refreshing
refreshing
refreshing
refreshing
refreshing
refreshing
refreshing
refreshing
refreshing
refreshing
refreshing